In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

train=pd.read_csv('train.csv', index_col=0)
test=pd.read_csv('test.csv', index_col=0)

#print(train['Attrition'].value_counts()) #查看label的取值分布，看是否均衡
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0) #处理Attrition字段，将label转换成数值
#print(train.isna().sum()) #查看数据是否有空值

#去掉没用的列(所有行取值都不同或都相同的列，即方差为0的列) 员工号码，标准工时(=80)
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

#可以用.corr()得到各特征的相关性矩阵，然后用heatmap展示，对于高相关性的特征可以只保留一个

# 对于分类特征进行特征值编码LabelEncoder
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)
train.to_csv('train_label_encoder.csv')

from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

#模型训练+预测(子页)


In [ ]:
#XGBOOST模型
import xgboost as xgb
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic',
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
}
# 使用DictVectorizer对离散值进行编码
dvec = DictVectorizer(sparse=False)
X_train = dvec.fit_transform(X_train.to_dict(orient='record'))
X_valid = dvec.transform(X_valid.to_dict(orient='record'))
temp_test = dvec.transform(test.to_dict(orient='record'))
# 进行DMatrix转换
train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(temp_test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)

In [ ]:
import xgboost as xgb

param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
        }

# 使用DictVectorizer对离散值进行编码
dvec = DictVectorizer(sparse=False)
X_train = dvec.fit_transform(X_train.to_dict(orient='record'))
X_valid = dvec.transform(X_valid.to_dict(orient='record'))
temp_test = dvec.transform(test.to_dict(orient='record'))

# 进行DMatrix转换
train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(temp_test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
y_pred = model.predict(test_data)
y_pred = [1 if x>=0.5 else 0 for x in y_pred]
print('XGBoost 预测结果', y_pred)
print('XGBoost 预测准确率：', accuracy_score(y_test, y_pred))

In [ ]:
test['Attrition']=predict

print(test['Attrition'])
test[['Attrition']].to_csv('submit.csv')
print('submit_lr.csv saved')

# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test[['Attrition']].to_csv('submit.csv')